In [1]:
import json
import numpy as np
import pandas as pd
import multiprocessing

from pymoo.optimize import minimize
from pymoo.core.callback import Callback
from pymoo.algorithms.moo.rvea import RVEA
from pymoo.core.problem import ElementwiseProblem, StarmapParallelization
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.pntx import SinglePointCrossover

In [2]:
# Read CSVs
df_classrooms = pd.read_csv("./datasets/clean/CaracterizaçãoDasSalasSmall.csv", sep=";", header=0)
df_classes = pd.read_csv("./datasets/clean/HorarioDeExemploSmall.csv", sep=";", header=0)

def get_characteristics(row):
  cols_with_x = [col for col in row.index if row[col] == "X"]
  return ", ".join(cols_with_x)

# Prepare df_classrooms
df_classrooms["caracteristicas"] = df_classrooms.apply(get_characteristics, axis=1)
df_classrooms = df_classrooms[["Nome sala", "Capacidade Normal", "caracteristicas"]]
df_classrooms = df_classrooms.rename(columns={"Nome sala":"Sala da aula", "Capacidade Normal":"Lotação", "caracteristicas":"Características reais da sala"})
print("df_classrooms len: ", len(df_classrooms))

# Prepare df_classes
df_classes["Sala da aula"] = ""
df_classes["Lotação"] = ""
df_classes["Características reais da sala"] = ""
print("df_classes len: ", len(df_classes))

df_classrooms len:  85
df_classes len:  5746


In [3]:
class ISCTE_CLASSES(ElementwiseProblem):
  def __init__(self, df_classes, df_classrooms, constraints=[500, 1300], **kwargs):
    self.df_classes = df_classes
    self.df_classrooms = df_classrooms
    self.constraints = constraints

    n_var = len(df_classes)

    xl = np.zeros(n_var)
    xu = np.array([len(df_classrooms) - 1] * len(df_classes))

    super().__init__(n_var=n_var, n_obj=3, n_ieq_constr=2, n_eq_constr=0, xl=xl, xu=xu, **kwargs)


  def _evaluate(self, x, out, *args, **kwargs):
    df_schedule = self.arr_to_df(x)
    schedule_eval = self.df_evaluate(df_schedule)

    superlotacao = schedule_eval[1]
    sobreposicao = schedule_eval[0]

    sobreposicao_const, superlotacao_const = self.constraints

    out["F"] = np.array(schedule_eval)
    out["G"] = [sobreposicao - sobreposicao_const, superlotacao - superlotacao_const]


  def arr_to_df(self, arr_schedule):
    df_schedule = self.df_classes.copy()

    for i, row in df_schedule.iterrows():
      if arr_schedule[i] >= 97:
        print(arr_schedule)

      classroom = self.df_classrooms.iloc[int(arr_schedule[i])]

      df_schedule.at[i, "Sala da aula"] = classroom.iloc[0]
      df_schedule.at[i, "Lotação"] = classroom.iloc[1]
      df_schedule.at[i, "Características reais da sala"] = classroom.iloc[2]

    df_schedule["Datetime Início"] = pd.to_datetime(df_schedule['Dia'] + ' ' + df_schedule['Início'], format='%d/%m/%Y %H:%M:%S')
    df_schedule["Datetime Fim"] = pd.to_datetime(df_schedule['Dia'] + ' ' + df_schedule['Fim'], format='%d/%m/%Y %H:%M:%S')
    df_schedule["Lotação - Inscritos no turno"] = df_schedule.apply(lambda x: int(x["Lotação"]) - int(x["Inscritos no turno"]), axis=1)
    
    return df_schedule


  def check_overlap(self, start, end, df):
    for _, row in df.iterrows():
      tmp_start = row["Datetime Início"]
      tmp_end = row["Datetime Fim"]

      if tmp_start <= start < tmp_end or tmp_start < end <= tmp_end:
        return True
      
    return False


  def df_evaluate(self, df):
    sobreposicao = 0
    superlotacao = (df["Lotação - Inscritos no turno"] < 0).sum()
    caracteristicas = df.apply(lambda row: row["Características da sala pedida para a aula"] not in row["Características reais da sala"], axis=1).sum()

    for i, row in df.iterrows():
      tmp_df = df.loc[i + 1:].copy()
      tmp_df = tmp_df[(tmp_df['Sala da aula'] == row['Sala da aula']) & (tmp_df['Dia'] == row['Dia'])].copy()
      
      if tmp_df.shape[0] > 0:
        if self.check_overlap(row["Datetime Início"], row["Datetime Fim"], tmp_df):
          sobreposicao += 1

    return [sobreposicao, superlotacao, caracteristicas]

In [4]:
class MyCallback(Callback):

  def __init__(self) -> None:
    super().__init__()


  def notify(self, algorithm):
    constraints = algorithm.problem.constraints

    for el in algorithm.pop.get("F"):
      constraints = self.update_constraints(constraints, el[0], el[1])

    algorithm.problem.constraints = constraints


  def update_constraints(self, constraints, sobreposicao, superlotacao):
    sobreposicao_const, superlotacao_const = constraints

    new_sobreposicao_const = sobreposicao if sobreposicao < sobreposicao_const else sobreposicao_const
    new_superlotacao_const = superlotacao if superlotacao < superlotacao_const else superlotacao_const

    return [new_sobreposicao_const, new_superlotacao_const]

In [5]:
# the number of process to be used
n_process = 6

# initialize the pool
pool = multiprocessing.Pool(n_process)

runner = StarmapParallelization(pool.starmap)

iscte_problem = ISCTE_CLASSES(df_classes, df_classrooms, elementwise_runner=runner)

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=13)

rvea = RVEA(
  pop_size=105,
  ref_dirs=ref_dirs,
  sampling=IntegerRandomSampling(),
  crossover=SinglePointCrossover(prob=1),
  mutation=PolynomialMutation(prob=1, eta=10, repair=RoundingRepair()),
  eliminate_duplicates=True
)

iscte_res = minimize(
  iscte_problem,
  rvea,
  seed=7,
  save_history=True,
  verbose=True,
  callback=MyCallback(),
  termination=('n_eval', 10500)
)

print('Threads:', iscte_res.exec_time)
# pool.close()

X = iscte_res.opt.get("X")
F = iscte_res.opt.get("F")

# print(X)
print(F)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      105 |    105 |  0.000000E+00 |  2.429524E+01 |             - |             -
     2 |      210 |    105 |  0.000000E+00 |  2.668571E+01 |  0.3960396040 |         nadir
     3 |      315 |    105 |  0.000000E+00 |  2.587619E+01 |  0.000000E+00 |             f
     4 |      420 |    105 |  0.000000E+00 |  2.512381E+01 |  0.000000E+00 |             f
     5 |      525 |    105 |  0.000000E+00 |  2.422857E+01 |  0.2173913043 |         ideal
     6 |      630 |    105 |  0.000000E+00 |  2.243810E+01 |  0.000000E+00 |             f
     7 |      735 |    105 |  0.000000E+00 |  2.213333E+01 |  0.000000E+00 |             f
     8 |      840 |    105 |  0.000000E+00 |  2.128571E+01 |  0.000000E+00 |             f
     9 |      945 |    105 |  0.000000E+00 |  2.107619E+01 |  0.000000E+00 |             f
    10 |     1050 |    105 |  0.000000E+00 |  2.061905E+01 |  0.000000E+00 |             f

In [6]:
pool.close()

In [11]:
print(iscte_res.history[98].opt.get("F"))

[[ 453. 1236. 2455.]
 [ 463. 1295. 2473.]
 [ 470. 1288. 2498.]
 [ 470. 1284. 2452.]
 [ 481. 1273. 2445.]
 [ 487. 1290. 2438.]
 [ 481. 1263. 2462.]
 [ 476. 1283. 2397.]
 [ 496. 1240. 2483.]
 [ 499. 1251. 2423.]
 [ 501. 1298. 2481.]
 [ 501. 1286. 2476.]
 [ 482. 1301. 2488.]
 [ 474. 1303. 2434.]
 [ 504. 1262. 2518.]
 [ 505. 1291. 2503.]
 [ 444. 1210. 2448.]
 [ 495. 1305. 2463.]
 [ 476. 1306. 2471.]
 [ 506. 1264. 2468.]
 [ 506. 1260. 2488.]
 [ 477. 1307. 2469.]
 [ 461. 1232. 2460.]
 [ 494. 1308. 2471.]
 [ 481. 1308. 2518.]
 [ 506. 1302. 2470.]
 [ 482. 1309. 2505.]
 [ 509. 1291. 2491.]
 [ 500. 1310. 2506.]
 [ 445. 1204. 2451.]
 [ 412. 1171. 2414.]
 [ 480. 1311. 2496.]
 [ 492. 1311. 2486.]
 [ 486. 1311. 2457.]
 [ 511. 1284. 2463.]
 [ 511. 1284. 2509.]
 [ 482. 1311. 2480.]
 [ 412. 1172. 2416.]
 [ 512. 1291. 2467.]
 [ 512. 1299. 2404.]
 [ 502. 1310. 2528.]
 [ 513. 1278. 2459.]
 [ 434. 1202. 2458.]
 [ 432. 1182. 2452.]
 [ 505. 1308. 2497.]
 [ 432. 1202. 2455.]
 [ 457. 1250. 2456.]
 [ 509. 1305.

In [8]:
for i in [0, 9, 24, 49, 74, 98]:
  score_json = json.dumps(iscte_res.history[i].opt.get("F").tolist(), indent=4)
  schedule_json = json.dumps(iscte_res.history[i].opt.get("X").tolist(), indent=4)

  with open(f"./results/final/RVEA_constraint/score_gen_{i+1}.json", "w") as outfile:
    outfile.write(score_json)
    
  with open(f"./results/final/RVEA_constraint/schedule_gen_{i+1}.json", "w") as outfile:
    outfile.write(schedule_json)

In [9]:
# result_dict = {"Score": F.tolist(), "Schedule": X.tolist()}

# results_json = json.dumps(result_dict, indent=4)

# with open("./results/RVEA_result_2_1.json", "w") as outfile:
#   outfile.write(results_json)